
## Physics-Informed Neural Network for the Lorenz 96 System

The Lorenz 96 system is a set of ordinary differential equations that is commonly used as a toy model in atmospheric science. It exhibits chaotic behavior, which makes it an interesting subject for study.

In this notebook, we aim to build a Physics-Informed Neural Network (PINN) for the Lorenz 96 system. The idea behind a PINN is to combine the strengths of physics-based modeling with data-driven machine learning. By informing the neural network about the underlying physical laws (in this case, the Lorenz 96 equations), we hope to improve the model's accuracy and interpretability.


In [7]:
import numpy as np
import tensorflow as tf
from scipy.integrate import odeint

In [2]:

# Echo State Network (ESN) for Reservoir Computing (RC)
# This class implements the reservoir computing approach using an Echo State Network. 
# Reservoir computing is a framework applied to recurrent neural networks where 
# the recurrent layer (the reservoir) is kept fixed, and only the readout weights are trained.

class RC_ESN:
    def __init__(self, input_dim, reservoir_size, output_dim, spectral_radius=0.95, sparsity=0.1):
        # Initialize reservoir weights and input weights with random values
        # Apply spectral radius to ensure the echo state property (ESP)
        
        # Reservoir weights
        self.W_res = np.random.rand(reservoir_size, reservoir_size) - 0.5
        eigenvalues = np.linalg.eigvals(self.W_res)
        self.W_res *= spectral_radius / max(abs(eigenvalues))
        
        # Input weights
        self.W_in = np.random.rand(reservoir_size, input_dim) * 2 - 1
        
        # Apply sparsity to the reservoir
        mask = np.random.rand(reservoir_size, reservoir_size) > sparsity
        self.W_res[mask] = 0
        
        # Set reservoir size, input and output dimensions
        self.reservoir_size = reservoir_size
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        # Initialize the reservoir state with zeros
        self.state = np.zeros(reservoir_size)
        self.W_out = None

    def _update(self, input_data):
        # Update the reservoir state using the input data and the current state
        # This is based on the hyperbolic tangent activation function
        self.state = np.tanh(np.dot(self.W_in, input_data) + np.dot(self.W_res, self.state))
        return self.state

    def fit(self, X, y, reg_coef=1e-6):
        # Train the readout weights (W_out) using Ridge Regression
        N = len(X)
        design_matrix = np.zeros((N, self.reservoir_size))
        
        # Update reservoir states for each input data
        for i in range(N):
            design_matrix[i] = self._update(X[i])
            
        # Compute the readout weights
        self.W_out = np.dot(np.linalg.inv(np.dot(design_matrix.T, design_matrix) + 
                                          reg_coef * np.eye(self.reservoir_size)), 
                            np.dot(design_matrix.T, y))

    def predict(self, X):
        # Predict the output for given input data using the trained readout weights
        N = len(X)
        output = np.zeros((N, self.output_dim))
        
        # Compute the output for each input data
        for i in range(N):
            reservoir_state = self._update(X[i])
            output[i] = np.dot(self.W_out.T, reservoir_state)
            
        return output



### Step 1: Define the Physics Constraints

The Lorenz 96 system is described by a set of ordinary differential equations (ODEs). These equations represent the physics constraints that any model of the system should adhere to. By expressing these ODEs in TensorFlow, we can later integrate them into our neural network model.


In [3]:


def lorenz_96_derivatives_tf(x, F=8):
    N = tf.shape(x)[0]
    dxdt = tf.zeros_like(x)
    for i in range(N):
        dxdt_i = (x[(i + 1) % N] - x[i - 2]) * x[i - 1] - x[i] + F
        dxdt = tf.tensor_scatter_nd_update(dxdt, [[i]], [dxdt_i])
    return dxdt



### Step 2: Design the Neural Network Architecture

To predict the derivatives of the Lorenz 96 system, we'll construct a neural network using TensorFlow. This neural network will be used in tandem with the physics constraints defined earlier. The aim is to train the neural network to approximate the dynamics of the Lorenz 96 system while ensuring that it adheres to the physical laws captured by the ODEs.


In [4]:

# Neural Network Architecture for Lorenz 96 System Prediction
# We construct a simple feedforward neural network with two hidden layers, each containing 64 neurons.
# The input and output sizes both match the dimension of the Lorenz 96 system, which is set to 'N'.

N = 5

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(N,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(N)
])

model.compile(optimizer='adam', loss='mse')
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                384       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 5)                 325       
                                                                 
Total params: 4869 (19.02 KB)
Trainable params: 4869 (19.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
esn = RC_ESN(input_dim=N, reservoir_size=100, output_dim=N)


### Step 3: Embed Physics Constraints as Loss Terms

To ensure that our neural network respects the physics of the Lorenz 96 system, we embed the physics constraints as additional loss terms. 

- `physics_loss`: Computes the mean squared error between the predicted derivatives from the neural network and those computed from the Lorenz 96 ODEs.
- `combined_loss`: Combines the standard mean squared error with the physics-based loss to guide the training of the neural network.


In [6]:
def physics_loss(y_true, y_pred):
    predicted_derivatives = lorenz_96_derivatives_tf(y_pred)
    return tf.reduce_mean(tf.square(predicted_derivatives - y_true))

def combined_loss(y_true, y_pred):
    mse_loss = tf.reduce_mean(tf.square(y_pred - y_true))
    return mse_loss + physics_loss(y_true, y_pred)

model.compile(optimizer='adam', loss=combined_loss)
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                384       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 5)                 325       
                                                                 
Total params: 4869 (19.02 KB)
Trainable params: 4869 (19.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________



### Step 4: Generate Training Data

To train our neural network, we require data that captures the dynamics of the Lorenz 96 system. In this step, we'll simulate the Lorenz 96 system to generate training samples. These samples will be used to train our neural network, ensuring that it learns to approximate the system's behavior while adhering to the underlying physics.


In [9]:
F = 8
# Initial state for generating training data
x0_train = F * np.ones(N)
x0_train[0] += 0.01  # Perturb the initial condition slightly

# Time points for generating training data
t_train = np.arange(0.0, 20.0, 0.01)  # For example, you can adjust the range and step size as needed

# Solve the differential equations to generate training data using the Lorenz 96 system's derivatives
x_train = odeint(lorenz_96_derivatives_tf, x0_train, t_train)

# Compute the derivatives for the training data
# This represents the change in the system state, which we aim to predict with our neural network
dxdt_train = np.array([lorenz_96_derivatives_tf(xi, 0) for xi in x_train])

# The input to our neural network will be the system's state at a given time
# The target (or label) will be the change in the system's state at the next time step
X_train = x_train[:-1]
y_train = x_train[1:] - x_train[:-1]



### Step 5: Train the Neural Network and the ESN

After generating the training data, our next step is to train both the neural network and the Echo State Network (ESN). The neural network will learn to approximate the dynamics of the Lorenz 96 system, while the ESN, a type of recurrent neural network, will be trained to predict the system's behavior based on the training samples.


In [10]:
# Train the Neural Network using the generated training data
# Note: You might need to adjust the number of epochs, batch size, etc. based on your dataset and problem complexity
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Train the Echo State Network (ESN) using the same training data
esn.fit(X_train, y_train)


Epoch 1/100
50/50 [==============================] - 2s 11ms/step - loss: 51.8018 - val_loss: 38.6252
Epoch 2/100
50/50 [==============================] - 0s 6ms/step - loss: 46.0607 - val_loss: 38.0058
Epoch 3/100
50/50 [==============================] - 0s 7ms/step - loss: 44.3478 - val_loss: 37.4306
Epoch 4/100
50/50 [==============================] - 0s 7ms/step - loss: 42.4575 - val_loss: 38.1986
Epoch 5/100
50/50 [==============================] - 0s 6ms/step - loss: 41.0003 - val_loss: 38.8619
Epoch 6/100
50/50 [==============================] - 0s 6ms/step - loss: 38.7665 - val_loss: 38.0394
Epoch 7/100
50/50 [==============================] - 0s 6ms/step - loss: 35.6703 - val_loss: 38.5635
Epoch 8/100
50/50 [==============================] - 0s 6ms/step - loss: 33.4525 - val_loss: 38.1524
Epoch 9/100
50/50 [==============================] - 0s 6ms/step - loss: 32.6589 - val_loss: 37.2331
Epoch 10/100
50/50 [==============================] - 0s 6ms/step - loss: 32.4973 - val_lo


### Step 6: Evaluate and Use the Hybrid Model

After training, it's essential to evaluate the model's performance on unseen data to validate its accuracy and generalization capabilities. By evaluating the model on a test dataset, we can gauge how well it has learned to predict the dynamics of the Lorenz 96 system.

Once validated, the trained PINN can be used for various tasks related to the Lorenz 96 system, such as forecasting, sensitivity analysis, or even control applications.


In [11]:
# Initial state for test data
x0_test = F * np.ones(N)
x0_test[0] += 0.02  # Slightly different initial condition from training

# Time points for test data
t_test = np.arange(0.0, 30.0, 0.01)

# Solve the differential equations for test data using the Lorenz 96 system's derivatives
x_test = odeint(lorenz_96_derivatives_tf, x0_test, t_test)

# Compute the derivatives for test data
dxdt_test = np.array([lorenz_96_derivatives_tf(xi, 0) for xi in x_test])

# Prepare test data for evaluation
X_test = x_test[:-1]
y_test = x_test[1:] - x_test[:-1]

# Evaluate the Neural Network on test data
nn_test_loss = model.evaluate(X_test, y_test)
print(f"Neural Network Test Loss: {nn_test_loss}")

# Use ESN to make predictions on test data
esn_predictions = esn.predict(X_test)

# Compute the Mean Squared Error for ESN predictions
esn_mse = ((y_test - esn_predictions) ** 2).mean()
print(f"ESN Mean Squared Error: {esn_mse}")


94/94 [==============================] - 0s 2ms/step - loss: 35.4622
Neural Network Test Loss: 35.46217346191406
ESN Mean Squared Error: 0.48590193511146823
